# 前処理

## インポート

In [ ]:
from fabric import Connection, Config

## 接続情報

In [ ]:
remote_host = '192.168.128.10'
remote_user = 'vagrant'
node_name   = 'ansible-ubuntu22'
key_file    = f'.vagrant/machines/{node_name}/hyperv/private_key'

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

## python 仮想環境用関数

In [ ]:
def run_py(c, cmd, venv="/opt/ansible-env/bin/activate", use_sudo=False, **kwargs):
    full_cmd = f"bash -lc 'source {venv} && {cmd}'"
    if use_sudo:
        return c.sudo(full_cmd, **kwargs)
    else:
        return c.run(full_cmd, **kwargs)


# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

In [ ]:
run_py(c,'ansible --version')

# 処理実行

## SSH接続確認

In [ ]:
for node in ["192.168.128.10", "192.168.128.11", "192.168.128.12"]:
    c.run(f"ssh -o StrictHostKeyChecking=no vagrant@{node} 'hostname'")

## hosts作成

In [ ]:
c.sudo('mkdir /etc/ansible')

In [ ]:
c.run('echo "[ubuntu]\n192.168.128.10 ansible_python_interpreter=/usr/bin/python3\n192.168.128.11 ansible_python_interpreter=/usr/bin/python3\n\n[rocky]\n192.168.128.12 ansible_python_interpreter=/usr/libexec/platform-python" | sudo tee /etc/ansible/hosts')

## ansibleテスト

In [ ]:
run_py(c,'ansible all -m ping')

In [ ]:
c.run('''echo "---
- hosts: all
  become: true
  tasks:
    - name: Create test file
      file:
        path: /home/vagrant/ansible_test_file
        state: touch
" > /home/vagrant/test.yml''')

In [ ]:
run_py(c,'ansible-playbook /home/vagrant/test.yml')

In [ ]:
for node in ["192.168.128.10", "192.168.128.11", "192.168.128.12"]:
    c.run(f"ssh vagrant@{node} 'ls -l /home/vagrant/ansible_test_file'")

In [ ]:
for node in ["192.168.128.10", "192.168.128.11", "192.168.128.12"]:
    c.run(f"ssh vagrant@{node} 'rm -f /home/vagrant/ansible_test_file'")

In [ ]:
c.close()